In [1]:
!pip install wandb

     |████████████████████████████████| 1.7 MB 5.1 MB/s 
     |████████████████████████████████| 97 kB 6.3 MB/s 
     |████████████████████████████████| 140 kB 54.5 MB/s 
     |████████████████████████████████| 180 kB 55.3 MB/s 
     |████████████████████████████████| 63 kB 1.1 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=6d48cd48c5f944e0633a1ef47f5aa10160b66b482e2c3aac3032794ea701cdf9
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=54ddd520897ca809b02ae553875e6c6c864a7c7134047e2ec5b46936536bfc33
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools


In [3]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
import random

# Launch 5 simulated experiments
for run in range(5):
  # 1️⃣ Start a new run to track this script
  wandb.init(
      # Set entity to specify your username or team name
      # ex: entity="carey",
      # Set the project where this run will be logged
      project="basic-intro", 
      # Track hyperparameters and run metadata
      config={
      "learning_rate": 0.02,
      "architecture": "CNN",
      "dataset": "CIFAR-100",})
  
  # This simple block simulates a training loop logging metrics
  offset = random.random() / 5
  for ii in range(2, 10):
      acc = 1 - 2 ** -ii - random.random() / ii - offset
      loss = 2 ** -ii + random.random() / ii + offset
      # 2️⃣ Log metrics from your script to W&B
      wandb.log({"acc": acc, "loss": loss})
      
  # Mark the run as finished
  wandb.finish()

wandb: Currently logged in as: smaxtoday (use `wandb login --relogin` to force relogin)


acc,▁▇▆▇█▇█▇
loss,█▆▂▃▁▁▂▂
acc,0.83009
loss,0.181


acc,▁▄▅▇█▆▇█
loss,█▃▂▂▁▂▁▁
acc,0.82246
loss,0.2402


acc,▁▃▄█▆▆▇█
loss,▅█▃▃▃▂▁▁
acc,0.85225
loss,0.18957


acc,▁▆▆▆▇▇▇█
loss,█▅▂▂▂▁▂▁
acc,0.95324
loss,0.06722


acc,▁▅▆▇▇▇██
loss,█▄▂▂▃▂▂▁
acc,0.9219
loss,0.03256


In [5]:
%wandb stacey/deep-drive/runs/1wyssjcx -h 720

In [6]:
import random

import numpy as np
import tensorflow as tf
from wandb.keras import WandbCallback

# Simple Keras Model

# Launch 20 experiments, trying different dropout rates
for run in range(20):
  # Start a run, tracking hyperparameters
  wandb.init(
      project="keras-intro",
      # Set entity to specify your username or team name
      # ex: entity="wandb",
      config={
          "layer_1": 512,
          "activation_1": "relu",
          "dropout": random.uniform(0.01, 0.80),
          "layer_2": 10,
          "activation_2": "softmax",
          "optimizer": "sgd",
          "loss": "sparse_categorical_crossentropy",
          "metric": "accuracy",
          "epoch": 6,
          "batch_size": 256
      })
  config = wandb.config

  # Get the data
  mnist = tf.keras.datasets.mnist
  (x_train, y_train), (x_test, y_test) = mnist.load_data()
  x_train, x_test = x_train / 255.0, x_test / 255.0
  x_train, y_train = x_train[::5], y_train[::5]  # Subset data for a faster demo
  x_test, y_test = x_test[::20], y_test[::20]
  labels = [str(digit) for digit in range(np.max(y_train) + 1)]

  # Build a model
  model = tf.keras.models.Sequential([
      tf.keras.layers.Flatten(input_shape=(28, 28)),
      tf.keras.layers.Dense(config.layer_1, activation=config.activation_1),
      tf.keras.layers.Dropout(config.dropout),
      tf.keras.layers.Dense(config.layer_2, activation=config.activation_2)
      ])

  model.compile(optimizer=config.optimizer,
                loss=config.loss,
                metrics=[config.metric]
                )

  # WandbCallback auto-saves all metrics from model.fit(), plus predictions on validation_data
  logging_callback = WandbCallback(log_evaluation=True)

  history = model.fit(x=x_train, y=y_train,
                      epochs=config.epoch,
                      batch_size=config.batch_size,
                      validation_data=(x_test, y_test),
                      callbacks=[logging_callback]
                      )

  # Mark the run as finished
  wandb.finish()

11501568/11490434 [==============================] - 0s 0us/step
Epoch 1/6
47/47 [==============================] - 4s 63ms/step - loss: 2.0924 - accuracy: 0.3236 - val_loss: 1.8016 - val_accuracy: 0.6220
Epoch 2/6
47/47 [==============================] - 2s 41ms/step - loss: 1.6453 - accuracy: 0.6423 - val_loss: 1.4408 - val_accuracy: 0.7320
Epoch 3/6
47/47 [==============================] - 2s 45ms/step - loss: 1.3334 - accuracy: 0.7379 - val_loss: 1.1839 - val_accuracy: 0.7760
Epoch 4/6
47/47 [==============================] - 2s 48ms/step - loss: 1.1161 - accuracy: 0.7790 - val_loss: 1.0060 - val_accuracy: 0.7880
Epoch 5/6
47/47 [==============================] - 2s 35ms/step - loss: 0.9663 - accuracy: 0.7995 - val_loss: 0.8829 - val_accuracy: 0.8180
Epoch 6/6
47/47 [==============================] - 1s 19ms/step - loss: 0.8572 - accuracy: 0.8169 - val_loss: 0.7938 - val_accuracy: 0.8220


accuracy,▁▆▇▇██
epoch,▁▂▄▅▇█
loss,█▅▄▂▂▁
val_accuracy,▁▅▆▇██
val_loss,█▅▄▂▂▁
accuracy,0.81692
best_epoch,5
best_val_loss,0.7938
epoch,5
loss,0.85725
val_accuracy,0.822


Epoch 1/6
47/47 [==============================] - 1s 22ms/step - loss: 2.1894 - accuracy: 0.2045 - val_loss: 1.9109 - val_accuracy: 0.5160
Epoch 2/6
47/47 [==============================] - 1s 17ms/step - loss: 1.7965 - accuracy: 0.4797 - val_loss: 1.5727 - val_accuracy: 0.6880
Epoch 3/6
47/47 [==============================] - 1s 17ms/step - loss: 1.5058 - accuracy: 0.6230 - val_loss: 1.3150 - val_accuracy: 0.7540
Epoch 4/6
47/47 [==============================] - 1s 17ms/step - loss: 1.2900 - accuracy: 0.6857 - val_loss: 1.1252 - val_accuracy: 0.7920
Epoch 5/6
47/47 [==============================] - 1s 18ms/step - loss: 1.1295 - accuracy: 0.7169 - val_loss: 0.9846 - val_accuracy: 0.8000
Epoch 6/6
47/47 [==============================] - 1s 19ms/step - loss: 1.0103 - accuracy: 0.7502 - val_loss: 0.8825 - val_accuracy: 0.8140


accuracy,▁▅▆▇██
epoch,▁▂▄▅▇█
loss,█▆▄▃▂▁
val_accuracy,▁▅▇▇██
val_loss,█▆▄▃▂▁
accuracy,0.75017
best_epoch,5
best_val_loss,0.88246
epoch,5
loss,1.01032
val_accuracy,0.814


Epoch 1/6
47/47 [==============================] - 1s 21ms/step - loss: 2.2856 - accuracy: 0.1768 - val_loss: 1.8738 - val_accuracy: 0.6160
Epoch 2/6
47/47 [==============================] - 1s 17ms/step - loss: 1.9037 - accuracy: 0.3507 - val_loss: 1.5777 - val_accuracy: 0.7120
Epoch 3/6
47/47 [==============================] - 1s 17ms/step - loss: 1.6491 - accuracy: 0.4742 - val_loss: 1.3534 - val_accuracy: 0.7600
Epoch 4/6
47/47 [==============================] - 1s 18ms/step - loss: 1.4717 - accuracy: 0.5490 - val_loss: 1.1799 - val_accuracy: 0.7920
Epoch 5/6
47/47 [==============================] - 1s 18ms/step - loss: 1.3256 - accuracy: 0.5997 - val_loss: 1.0471 - val_accuracy: 0.8040
Epoch 6/6
47/47 [==============================] - 1s 18ms/step - loss: 1.2093 - accuracy: 0.6395 - val_loss: 0.9444 - val_accuracy: 0.8080


...

accuracy,▁▄▆▇▇█
epoch,▁▂▄▅▇█
loss,█▆▄▃▂▁
val_accuracy,▁▅▇███
val_loss,█▆▄▃▂▁
accuracy,0.67875
best_epoch,5
best_val_loss,0.90639
epoch,5
loss,1.12678
val_accuracy,0.8


Epoch 1/6
47/47 [==============================] - 1s 20ms/step - loss: 2.1039 - accuracy: 0.3424 - val_loss: 1.8647 - val_accuracy: 0.5380
Epoch 2/6
47/47 [==============================] - 1s 16ms/step - loss: 1.6852 - accuracy: 0.6604 - val_loss: 1.5084 - val_accuracy: 0.7100
Epoch 3/6
47/47 [==============================] - 1s 17ms/step - loss: 1.3778 - accuracy: 0.7460 - val_loss: 1.2428 - val_accuracy: 0.7520
Epoch 4/6
47/47 [==============================] - 1s 16ms/step - loss: 1.1533 - accuracy: 0.7870 - val_loss: 1.0555 - val_accuracy: 0.7700
Epoch 5/6
47/47 [==============================] - 1s 16ms/step - loss: 0.9934 - accuracy: 0.8104 - val_loss: 0.9219 - val_accuracy: 0.7900
Epoch 6/6
47/47 [==============================] - 1s 16ms/step - loss: 0.8779 - accuracy: 0.8235 - val_loss: 0.8263 - val_accuracy: 0.8040


accuracy,▁▆▇▇██
epoch,▁▂▄▅▇█
loss,█▆▄▃▂▁
val_accuracy,▁▆▇▇██
val_loss,█▆▄▃▂▁
accuracy,0.8235
best_epoch,5
best_val_loss,0.82627
epoch,5
loss,0.87792
val_accuracy,0.804


Epoch 1/6
47/47 [==============================] - 1s 19ms/step - loss: 2.1286 - accuracy: 0.2682 - val_loss: 1.8512 - val_accuracy: 0.6040
Epoch 2/6
47/47 [==============================] - 1s 17ms/step - loss: 1.7112 - accuracy: 0.5595 - val_loss: 1.4986 - val_accuracy: 0.7120
Epoch 3/6
47/47 [==============================] - 1s 16ms/step - loss: 1.4157 - accuracy: 0.6708 - val_loss: 1.2405 - val_accuracy: 0.7520
Epoch 4/6
47/47 [==============================] - 1s 16ms/step - loss: 1.2006 - accuracy: 0.7272 - val_loss: 1.0567 - val_accuracy: 0.7760
Epoch 5/6
47/47 [==============================] - 1s 16ms/step - loss: 1.0456 - accuracy: 0.7574 - val_loss: 0.9267 - val_accuracy: 0.7980
Epoch 6/6
47/47 [==============================] - 1s 16ms/step - loss: 0.9326 - accuracy: 0.7810 - val_loss: 0.8316 - val_accuracy: 0.8100


accuracy,▁▅▆▇██
epoch,▁▂▄▅▇█
loss,█▆▄▃▂▁
val_accuracy,▁▅▆▇██
val_loss,█▆▄▃▂▁
accuracy,0.781
best_epoch,5
best_val_loss,0.83162
epoch,5
loss,0.93261
val_accuracy,0.81
